In [10]:
import os
import pandas as pd

city        = 'bristol'
outroot     = '../ProcessedData/' + city 

users_friend_geo   = outroot + '/networks/' + city + '_users_geo_networkmeasures.csv'
users_friend_topo  = outroot + '/networks/' + city + '_users_topo_networkmeasures.csv'
users_sim          = outroot + '/networks/' + city + '_users_topo_networkmeasures.csv'
venues_sim         = outroot + '/networks/' + city + '_venues_sim_geo_networkmeasures.csv'



df_friend_geo  =  pd.read_csv(users_friend_geo,index_col=0) 

#for line in open(users_friend_geo):
#    print line

print (df_friend_geo.keys())


Index([u'clustering', u'closeness_geo', u'geo_stdev_of_ego', u'pagerank_geo',
       u'degree', u'constraint', u'closeness', u'triangle_size',
       u'eigenvector_geo', u'geo_size_of_ego', u'betweenness', u'pagerank',
       u'strength_geo', u'constraint_geo', u'egosize', u'betweenness_geo',
       u'clustering_geo', u'eigenvector', u'social_stretch'],
      dtype='object')


In [ ]:
'''
Created on May 29, 2018
CODE COMES WITH NO WARRANTY, I HAVEN'T TESTED IT.
But it gives you a hint on what are the packages to use and the overall procedure
@author: lajel
'''

import bz2
from rtree import index
import shapefile


'''
Extracts shapes from the shapefile
@param shapefile: the name of the shapefile
'''
def extract_shapes(shpfile):
    shp = shapefile.Reader(shpfile) #open the shapefile
    all_shapes = shp.shapes() # get all the polygons
    all_records = shp.records()
    return all_shapes, all_records

'''
Builds a spatial index out of coordinates
@param coords: list of triples (point_id, lat, lon)
'''
def build_rtree_index(coords):
    idx= index.Index()
    for coord in coords:
        point_id, lat, lon = coord
        idx.insert(point_id, (lon, lat, lon, lat))
    return idx

'''
Matches points to areas.
@param idx: the spatial index
@param all_shapes: the shapes
@param all_shapes: the records
'''
def get_pid2area(idx,all_shapes,all_records,outfile):
    point2area = {}
    for i in range(0,len(all_shapes)):
        area_id = all_records[i][1]
        shp = all_shapes[i]
        minx, miny, maxx, maxy = shp.bbox
        point_ids = list(idx.intersection((minx, miny, maxx, maxy)))
        for pid in point_ids:
            point2area[pid] = area_id
    if outfile is not None:
        fout = bz2.open(outfile,'wt',encoding='utf-8')
        for pid in point2area:
            fout.write('%s,%s\n'%(pid,point2area[pid]))
        fout.close()
    return point2area

if __name__ == '__main__':
    shpfile = 'cb_2017_us_cbsa_500k//cb_2017_us_cbsa_500k.shp'
    coordinates = [(2, 42.344859, -94.083647), (3, 43.291242, -85.472008), (4,33.312953, -103.731874)]
    all_shapes, all_records = extract_shapes(shpfile)
    index = build_rtree_index(coordinates)
    print(index)
    point2area = get_pid2area(index, all_shapes, all_records, 'output_file')
    print (point2area)
    
    